In [2]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize
import ast

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

In [3]:
# api_key 저장
api_key = 'RGAPI-ca44b272-60ec-4589-8980-8f28e65ca212' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

##### match data each participants 불러오기

In [4]:
match_df = pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/game_results_of_each_participants.csv')

match_df.head(1)

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,augments,traits,units
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,"['TFT9_Augment_IronAssets', 'TFT9_Augment_YouHaveMyBow', 'TFT6_Augment_PortableForgePlusPlus']","[{'name': 'TFT11_Altruist', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Behemoth', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 3}, {'name': 'TFT11_Bruiser', 'num_units': 4, 'style': 2, 'tier_current': 2, 'tier_total': 4}, {'name': 'TFT11_Exalted', 'num_units': 3, 'style': 2, 'tier_current': 1, 'tier_total': 2}, {'name': 'TFT11_Ghostly', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_InkShadow', 'num_units': 2, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Mythic', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_SpiritWalker', 'num_units': 1, 'style': 5, 'tier_current': 1, 'tier_total': 1}, {'name': 'TFT11_Storyweaver', 'num_units': 3, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Trickshot', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 2}, {'name': 'TFT11_Umbral', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}]","[{'character_id': 'TFT11_Sivir', 'itemNames': [], 'name': '', 'rarity': 0, 'tier': 2}, {'character_id': 'TFT11_Riven', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Shen', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 1}, {'character_id': 'TFT11_TahmKench', 'itemNames': ['TFT4_Item_OrnnRanduinsSanctum'], 'name': '', 'rarity': 2, 'tier': 2}, {'character_id': 'TFT11_Galio', 'itemNames': ['TFT_Item_Crownguard', 'TFT_Item_WarmogsArmor', 'TFT_Item_AdaptiveHelm'], 'name': '', 'rarity': 4, 'tier': 1}, {'character_id': 'TFT11_Kaisa', 'itemNames': ['TFT_Item_GuinsoosRageblade', 'TFT_Item_RapidFireCannon', 'TFT9_Item_OrnnHorizonFocus'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Sylas', 'itemNames': ['TFT_Item_Bloodthirster', 'TFT_Item_UnstableConcoction', 'TFT_Item_LocketOfTheIronSolari'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Udyr', 'itemNames': [], 'name': '', 'rarity': 6, 'tier': 1}, {'character_id': 'TFT11_Kayle', 'itemNames': ['TFT11_Storyweaver3_Support'], 'name': '', 'rarity': 9, 'tier': 1}]"


##### 불필요한 augments, traits 컬럼 drop 하기

In [5]:
match_df.drop(['augments', 'traits'], axis=1, inplace=True)

In [6]:
match_df.head(1)

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,units
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,"[{'character_id': 'TFT11_Sivir', 'itemNames': [], 'name': '', 'rarity': 0, 'tier': 2}, {'character_id': 'TFT11_Riven', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Shen', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 1}, {'character_id': 'TFT11_TahmKench', 'itemNames': ['TFT4_Item_OrnnRanduinsSanctum'], 'name': '', 'rarity': 2, 'tier': 2}, {'character_id': 'TFT11_Galio', 'itemNames': ['TFT_Item_Crownguard', 'TFT_Item_WarmogsArmor', 'TFT_Item_AdaptiveHelm'], 'name': '', 'rarity': 4, 'tier': 1}, {'character_id': 'TFT11_Kaisa', 'itemNames': ['TFT_Item_GuinsoosRageblade', 'TFT_Item_RapidFireCannon', 'TFT9_Item_OrnnHorizonFocus'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Sylas', 'itemNames': ['TFT_Item_Bloodthirster', 'TFT_Item_UnstableConcoction', 'TFT_Item_LocketOfTheIronSolari'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Udyr', 'itemNames': [], 'name': '', 'rarity': 6, 'tier': 1}, {'character_id': 'TFT11_Kayle', 'itemNames': ['TFT11_Storyweaver3_Support'], 'name': '', 'rarity': 9, 'tier': 1}]"


In [7]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9288 entries, 0 to 9287
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameId         9288 non-null   object 
 1   game_datetime  9288 non-null   int64  
 2   game_length    9288 non-null   float64
 3   game_version   9288 non-null   object 
 4   puuid          9288 non-null   object 
 5   placement      9288 non-null   int64  
 6   last_round     9288 non-null   int64  
 7   level          9288 non-null   int64  
 8   units          9288 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 653.2+ KB


##### UNITS_TIERS 테이블 생성 - 유닛명을 컬럼으로 유닛의 최종 별 갯수를 값을 가지는 컬럼 생성

In [8]:
import ast

# units 컬럼의 문자열을 리스트로 변환
match_df['units'] = match_df['units'].apply(ast.literal_eval)

# character_id 및 tier 값 추출
characters = {}
for unit_list in match_df['units']:
    for unit in unit_list:
        character_id = unit['character_id']
        tier = unit['tier']
        if character_id not in characters:
            characters[character_id] = tier
        else:
            characters[character_id] = max(characters[character_id], tier)

# 새로운 컬럼 생성 및 초기화
for character_id, tier in characters.items():
    match_df[character_id] = 0

# character_id에 해당하는 컬럼의 tier 값 설정
for index, unit_list in enumerate(match_df['units']):
    for unit in unit_list:
        character_id = unit['character_id']
        tier = unit['tier']
        match_df.at[index, character_id] = tier

# units 컬럼 제거
match_df = match_df.drop(columns=['units'])

match_df.head()

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,TFT11_Sivir,TFT11_Riven,...,TFT11_Syndra,TFT11_FortuneYord,TFT11_Darius,TFT11_Aphelios,TFT11_Thresh,TFT11_ChoGath,TFT11_Malphite,TFT11_Garen,TFT11_Teemo,TFT11_Xayah
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,2,2,...,0,0,0,0,0,0,0,0,0,0
1,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ,1,40,9,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,iM__viRIlCBc3VBF9p0BK9a58MsQTrB6Svi4asmtcw4vxkVZgPWgAB3ZY98VJHwbV2YhWUTzZr_XEQ,4,34,9,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,XwRIqTXBnBYgAU3Jct9FIDtrIOy01HBn4PUM6BecUex8D2MRFJlhDu2fcAvohMYPQrRGJK65ccfeCQ,8,27,9,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,bAUTBjhuA00n1167MUmonjyMgetU3Dv1-5IGNUqWwWt-59C4f_W2hhtPLqZ5KQTNdoQCjExuuGszZg,6,28,8,0,1,...,0,0,0,0,0,0,0,0,0,0


##### 각 참가자가 최종적으로 사용한 챔피언의 이름들만 리스트로 구성되어있는 Combination 컬럼 생성

In [9]:
# Combination 컬럼 생성
match_df['Combination'] = match_df.filter(like='TFT11').apply(lambda x: ', '.join(x.index[x > 0]), axis=1)

match_df.head()

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,TFT11_Sivir,TFT11_Riven,...,TFT11_FortuneYord,TFT11_Darius,TFT11_Aphelios,TFT11_Thresh,TFT11_ChoGath,TFT11_Malphite,TFT11_Garen,TFT11_Teemo,TFT11_Xayah,Combination
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,2,2,...,0,0,0,0,0,0,0,0,0,"TFT11_Sivir, TFT11_Riven, TFT11_Shen, TFT11_TahmKench, TFT11_Galio, TFT11_Kaisa, TFT11_Sylas, TFT11_Udyr, TFT11_Kayle"
1,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ,1,40,9,0,0,...,0,0,0,0,0,0,0,0,0,"TFT11_Caitlyn, TFT11_RekSai, TFT11_Jax, TFT11_Aatrox, TFT11_Kindred, TFT11_Senna, TFT11_Gnar, TFT11_Ornn, TFT11_Azir"
2,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,iM__viRIlCBc3VBF9p0BK9a58MsQTrB6Svi4asmtcw4vxkVZgPWgAB3ZY98VJHwbV2YhWUTzZr_XEQ,4,34,9,0,0,...,0,0,0,0,0,0,0,0,0,"TFT11_Udyr, TFT11_Ornn, TFT11_Azir, TFT11_Janna, TFT11_Soraka, TFT11_Diana, TFT11_Hwei, TFT11_Rakan, TFT11_WuKong"
3,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,XwRIqTXBnBYgAU3Jct9FIDtrIOy01HBn4PUM6BecUex8D2MRFJlhDu2fcAvohMYPQrRGJK65ccfeCQ,8,27,9,0,0,...,0,0,0,0,0,0,0,0,0,"TFT11_TahmKench, TFT11_Sylas, TFT11_Udyr, TFT11_Ornn, TFT11_Hwei, TFT11_Rakan, TFT11_Annie, TFT11_Lillia, TFT11_Lissandra"
4,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,bAUTBjhuA00n1167MUmonjyMgetU3Dv1-5IGNUqWwWt-59C4f_W2hhtPLqZ5KQTNdoQCjExuuGszZg,6,28,8,0,1,...,0,0,0,0,0,0,0,0,0,"TFT11_Riven, TFT11_Galio, TFT11_Udyr, TFT11_Kayle, TFT11_Ornn, TFT11_Janna, TFT11_Diana, TFT11_Zyra, TFT11_Morgana"


##### 테이블 저장

In [10]:
match_df.to_csv('units_tiers_and_combination.csv', index=False)

In [11]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9288 entries, 0 to 9287
Data columns (total 70 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gameId             9288 non-null   object 
 1   game_datetime      9288 non-null   int64  
 2   game_length        9288 non-null   float64
 3   game_version       9288 non-null   object 
 4   puuid              9288 non-null   object 
 5   placement          9288 non-null   int64  
 6   last_round         9288 non-null   int64  
 7   level              9288 non-null   int64  
 8   TFT11_Sivir        9288 non-null   int64  
 9   TFT11_Riven        9288 non-null   int64  
 10  TFT11_Shen         9288 non-null   int64  
 11  TFT11_TahmKench    9288 non-null   int64  
 12  TFT11_Galio        9288 non-null   int64  
 13  TFT11_Kaisa        9288 non-null   int64  
 14  TFT11_Sylas        9288 non-null   int64  
 15  TFT11_Udyr         9288 non-null   int64  
 16  TFT11_Kayle        9288 

In [12]:
match_df.T

,0,1,2,3,4,5,6,7,8,9,...,9278,9279,9280,9281,9282,9283,9284,9285,9286,9287
gameId,KR_7005435650,KR_7005435650,KR_7005435650,KR_7005435650,KR_7005435650,KR_7005435650,KR_7005435650,KR_7005435650,KR_7005408175,KR_7005408175,...,KR_7005149778,KR_7005149778,KR_7005139736,KR_7005139736,KR_7005139736,KR_7005139736,KR_7005139736,KR_7005139736,KR_7005139736,KR_7005139736
game_datetime,1711611092537,1711611092537,1711611092537,1711611092537,1711611092537,1711611092537,1711611092537,1711611092537,1711608635465,1711608635465,...,1711574024327,1711574024327,1711571877752,1711571877752,1711571877752,1711571877752,1711571877752,1711571877752,1711571877752,1711571877752
game_length,2339.676514,2339.676514,2339.676514,2339.676514,2339.676514,2339.676514,2339.676514,2339.676514,2161.773926,2161.773926,...,2346.074707,2346.074707,2173.421143,2173.421143,2173.421143,2173.421143,2173.421143,2173.421143,2173.421143,2173.421143
game_version,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,...,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>
puuid,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ,iM__viRIlCBc3VBF9p0BK9a58MsQTrB6Svi4asmtcw4vxkVZgPWgAB3ZY98VJHwbV2YhWUTzZr_XEQ,XwRIqTXBnBYgAU3Jct9FIDtrIOy01HBn4PUM6BecUex8D2MRFJlhDu2fcAvohMYPQrRGJK65ccfeCQ,bAUTBjhuA00n1167MUmonjyMgetU3Dv1-5IGNUqWwWt-59C4f_W2hhtPLqZ5KQTNdoQCjExuuGszZg,rx1nmyxEddDyQujL2nvjfiHV_LDcBEo-zyf16l3BFnXQExOMoB6tNss-WGlc_GRgcac2tg1GhDgBhg,dMhhqa_djjUT5P2jeQePRwhWzHVNKrf35C-z45JPsSKvfREWIhtlWUGHWke8oO3pf3ADGHpBABqL6g,yAVc-bTbi86qYF3ePtvTr0DTW367seRmnhXqPLIMwmw9xLZWbrw9uIX2wSTodseM41RIG-UnnGajOw,_tFZFQDxr3ipe-amA8IMZiUugmCq6rhTN9YrLPthGkJSnh1TbAB9j1v4_pUGvH7UWmwnNM_jUtDhAw,_VGTHEw4TgYAsfkDjFEvIk0FDfn4MMvItgLDd-NAq-lEKlTmv4oZ9TUrlvMGui-oiZvdi07yWsWOKg,...,vONPZNRf3tyDkd_mWk-0nDUPAgDJb6gt6qI_9qKfiXIL7U1WnibUtiWE1Mx5uIM3OZYog3SoT8fPsw,ZWNe7oOLcVQzKNoExAYXKQm_oZr6KsjEXji5ilpOOsg2efr-Tjl5Xe-xI6fXwpzyI7zEWDMdTZPRHQ,Vhs9Vq-UUa2_yiovh0mAgFJYQ8EUY5mfu_fMFJPttBTnCI0WqZNZMu9kiHVV3AUM_S3vbwxqIq1sRg,isHLordKTqogFpqDZ5fCHX8FsKdAPFLYZxPVb5-MeQuXV7qtso5qhL2bYU7ay4L3cU6pGoPgetcYXw,ci4IhQ-c9KUs-dkyNeqhitxs-7b5VGWZunymMHxUdooDZZ4cZ0L6x0mBX2VgPWCSi3No3H0OLN-h6g,sMD7iMyWZ2ztcGvcYyXf92yYF6ui0ipF8UInDkKm6wMnJc7abC_rkU3WJokvCuiO4Rln3ltQCc3xTw,4W5dMgFaI2qWXtzftLCjec16RypS_e_0BaHIynnzUApIEjpzmow5edpCgoq2Z7R4wwe8AhejYgf_sw,m-WJmbLS17zaoDDdN1NjqRMGuXkZChReSv8-36lQ32zgt2-TJU4Noj7CY246rk6qA0WmYVOE4kTogQ,eJBg1EVkQRS4fXbZKDAkyuQuYzL-kF1Ps7NOxHuwiAQ29VZgBEi20Je7sbVtqV-m-Y6ArWw2Hck5Iw,H7I8YjZ_H49ijjt7z4b6CZKJ0up1ug51YNS_PUhLR4p6cUAbQf0Vr91bpmtC2fQ4_DTZj_Hxl5OTlg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [16]:
match_df.groupby(['Combination']).nunique()

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,TFT11_Sivir,TFT11_Riven,...,TFT11_Syndra,TFT11_FortuneYord,TFT11_Darius,TFT11_Aphelios,TFT11_Thresh,TFT11_ChoGath,TFT11_Malphite,TFT11_Garen,TFT11_Teemo,TFT11_Xayah
Combination,,,,,,,,,,,,,,,,,,,,,
,3,3,3,1,3,3,3,3,1,1,...,1,1,1,1,1,1,1,1,1,1
"TFT11_Aatrox, TFT11_Azir, TFT11_Neeko, TFT11_Illaoi, TFT11_Ahri, TFT11_Lux, TFT11_Amumu, TFT11_Syndra",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"TFT11_Aatrox, TFT11_Diana, TFT11_Rakan, TFT11_WuKong, TFT11_Lissandra, TFT11_Zyra, TFT11_Morgana, TFT11_Illaoi",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"TFT11_Aatrox, TFT11_Diana, TFT11_WuKong, TFT11_Zyra, TFT11_Morgana, TFT11_Kayn, TFT11_Illaoi, TFT11_Nautilus",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"TFT11_Aatrox, TFT11_Janna, TFT11_Diana, TFT11_Rakan, TFT11_WuKong, TFT11_Annie, TFT11_Zyra, TFT11_Morgana, TFT11_Kayn, TFT11_LeeSin",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"TFT11_Zoe, TFT11_Illaoi, TFT11_Lux, TFT11_Amumu, TFT11_Syndra",3,3,3,1,3,3,3,2,1,1,...,2,1,1,1,1,1,1,1,1,1
"TFT11_Zoe, TFT11_Illaoi, TFT11_Nautilus, TFT11_Sett, TFT11_Ahri, TFT11_Lux, TFT11_Amumu, TFT11_Syndra",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"TFT11_Zoe, TFT11_Illaoi, TFT11_Nautilus, TFT11_Sett, TFT11_Ahri, TFT11_Lux, TFT11_Amumu, TFT11_Syndra, TFT11_Thresh",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
